In [1]:
a = [1.0, 2.0, 1.0]

In [2]:
a[0]

1.0

In [3]:
a[2] = 3.0
a

[1.0, 2.0, 3.0]

# 첫텐서 만들어보기

In [4]:
import torch
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [5]:
a[1]

tensor(1.)

In [6]:
float(a[1])

1.0

In [7]:
a[2] = 2.0
a

tensor([1., 1., 2.])

In [8]:
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

points

tensor([4., 1., 5., 3., 2., 1.])

In [9]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [10]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [11]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [12]:
points.shape

torch.Size([3, 2])

In [13]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [14]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [15]:
points[0, 1]

tensor(1.)

In [16]:
points[0]

tensor([4., 1.])

# 텐서 인덱싱

In [17]:
some_list = list(range(6))
some_list[:] # 리스트의 모든 요소

[0, 1, 2, 3, 4, 5]

In [18]:
print(some_list[1:4]) # 1번 인덱스 ~ 4번 인덱스 전
print(some_list[1:]) # 1번 인덱스 부터 끝까지
print(some_list[:4]) # 첫 번째 요소부터 4번 인덱스 전까지
print(some_list[:-1]) # 리스트의 첫번째 요소부터 마지막 요소 바로 앞까지
print(some_list[1:4:2]) # 1번 인덱스부터 4번 인덱스까지, 두 단계씩 건너뛰면서

[1, 2, 3]
[1, 2, 3, 4, 5]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[1, 3]


In [19]:
print(points[1:]) # 첫번째 이후 모든 행에 대해, 암묵적으로 모든 열 포함
print(points[1:, :]) # 첫번째 이후 모든 행에 대해, 명시적으로 모든 열 포함
print(points[1:, 0]) # 첫 번째 이후 모든 행에 대해 첫째 열만
print(points[None]) # 길이가 1인 차원 추가

tensor([[5., 3.],
        [2., 1.]])
tensor([[5., 3.],
        [2., 1.]])
tensor([5., 2.])
tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])


# 이름이 있는 텐서

In [20]:
img_t = torch.randn(3, 5, 5) # 각각이 [채널 크기, 행 크기, 열 크기]가 됨
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [21]:
batch_t = torch.randn(2, 3, 5, 5) # [채널 크기, 행 크기, 열 크기]

In [22]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [23]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [24]:
img_gray_weighted_fancy = torch.einsum('...chw, c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw, c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [25]:
# 이름 지정이 가능한 텐서
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names = ['channels'])
weights_named

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1463.)
  


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [26]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named:', img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [27]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [28]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [29]:
# 이름이 다른 차원을 결합하려면 오류가 발생
# gray_named = (img_named[..., :3] * weights_named).sum("channels")

In [30]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

# dtype으로 숫자 정하기
- torch.float32 혹은 torch.float : 32비트 단정밀도 부동소수점
- torch.float64 혹은 torch.double : 64비트 배정밀도 부동소수점
- torch.float16 혹은 torch.half : 16비트 반정밀도 부동소수점
- torch.int8 : 부호 있는 8비트 정수
- torch.uint8 : 부호 없는 8비트 정수
- torch.int16 혹은 torch.short : 부호 있는 16비트 정수
- torch.int32 혹은 torch.int : 부호 있는 32비트 정수
- torch.int64 혹은 torch.long : 부호 있는 64비트 정수
- torch.bool : 불리언

* 텐서의 기본 데이터 타입은 32비트 부동소수점이다.

In [31]:
# 텐서의 dtype 속성 관리
double_points = torch.ones(10, 2, dtype = torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype = torch.short)

In [32]:
short_points.dtype

torch.int16

In [33]:
double_points = torch.zeros(10,2).double()
short_points = torch.ones(10, 2).short()

In [34]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype = torch.short)

In [35]:
points_64 = torch.rand(5, dtype = torch.double)
points_short = points_64.to(torch.short)
points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

In [36]:
# 텐서 API
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [37]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [38]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [39]:
points_storage = points.storage()
points_storage[0]

4.0

In [40]:
points.storage()[1]

1.0

In [41]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

In [42]:
# 저장된 값을 수정하기 : 텐서 내부 연산
# 입력된 텐서의 모든 요소를 0으로 바꾼다.
a = torch.ones(3,2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [43]:
points = torch.tensor([[4.0, 1.0, 5.0], [5.0, 3.0, 4.0], [2.0, 1.0, 0.0], [2.0, 5.0, 0.0]])
second_point = points[3]
second_point.storage_offset() # 몇번째 요소인지

9

In [44]:
second_point.size()

torch.Size([3])

In [45]:
second_point.shape

torch.Size([3])

In [46]:
# 각 차원에서 인덱스를 하나 증가했을 때 저장 공간상에서 몇개 요소를 건너뛰어야 하는지 값을 가짐
points.stride()

(3, 1)

In [47]:
second_point = points[1]
second_point.size()

torch.Size([3])

In [48]:
second_point.storage_offset()

3

In [49]:
second_point.stride()

(1,)

In [50]:
points.stride()

(3, 1)

In [51]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [52]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [53]:
# 복사 없이 텐서 전치하기
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [54]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [55]:
id(points.storage()) == id(points_t.storage())

False

In [56]:
points.stride()

(2, 1)

In [57]:
points_t.stride()

(1, 2)

In [58]:
# 더 높은 차원에서의 전치 연산
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape

torch.Size([3, 4, 5])

In [59]:
transpose_t.shape

torch.Size([5, 4, 3])

In [60]:
some_t.stride()

(20, 5, 1)

In [61]:
transpose_t.stride()

(1, 5, 20)

In [62]:
# 가장 오른쪽 차원에서 시작해서 증가되는 형태의 텐서는 CPU 접근 효율이 좋음
print(points.is_contiguous())
print(points_t.is_contiguous())

True
False


In [63]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [64]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [65]:
points_t.stride()

(1, 2)

In [66]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [67]:
points_t_cont.stride()

(3, 1)

In [68]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]

In [71]:
# 텐서를 GPU로 옮기기
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device = 'cuda')

points_gpu = points.to(device = 'cuda')

In [73]:
points_gpu = points.to(device = 'cuda:0')

In [74]:
points = 2 * points
points_gpu = 2 * points.to(device = 'cuda')

In [75]:
points_gpu = points_gpu + 4

In [76]:
points_cpu = points_gpu.to(device = 'cpu')

In [77]:
points_gpu = points.cuda()
points_gpu = points.cuda(0)
points_cpu = points_gpu.cpu()

In [78]:
# 넘파이 호환
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [79]:
points = torch.from_numpy(points_np)

In [87]:
problem = torch.tensor([list(range(9))])
print(problem.size())
print(problem.stride())
print(problem.storage_offset())

torch.Size([1, 9])
(9, 1)
0


In [94]:
b = problem.view([3, 3])
a

In [95]:
id(a.storage()) == id(problem.storage())

True

In [102]:
c = b[1:, 1:]
c

tensor([[4, 5],
        [7, 8]])

In [103]:
print(c.size())
print(c.stride())
print(c.storage_offset())

torch.Size([2, 2])
(3, 1)
4


In [105]:
torch.cos(problem[0])

tensor([ 1.0000,  0.5403, -0.4161, -0.9900, -0.6536,  0.2837,  0.9602,  0.7539,
        -0.1455])